In [3]:
import warnings
warnings.filterwarnings("ignore", category=DeprecationWarning) 
%pdb off

Automatic pdb calling has been turned OFF


In [4]:
import sys
sys.path.append('../')
import src


In [5]:
import lightgbm as lgb
import pandas as pd
import numpy as np

from sklearn.model_selection import train_test_split
from sklearn.externals import joblib

In [6]:
from src.models.model import Model
from src.features.build_features import Vectorizer, alphanum_cont
from src.data.make_df import get_df

In [7]:
df = get_df()

In [8]:
X = df['code']
y = df['language']

In [9]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2)

In [10]:
vec1 = Vectorizer(tokenizer='char', build_hash_dict=True, n_features=2**10, n_gram=(2,))
model1 = Model(lgb.LGBMClassifier(), vec1)
%time model1.fit(X_train, y_train)
model1.evaluate(X_test, y_test)

CPU times: user 3min 31s, sys: 1 s, total: 3min 32s
Wall time: 1min 15s


/home/klui/.local/lib/python3.7/site-packages/sklearn/preprocessing/label.py:151: DeprecationWarning: The truth value of an empty array is ambiguous. Returning False, but in future this will result in an error. Use `array.size > 0` to check that an array is not empty.
  if diff:


0.9470649895178197

In [11]:
vec2 = Vectorizer(tokenizer='char', build_hash_dict=True, n_features=2**10, n_gram=(2,), hash_seed=1)
model2 = Model(lgb.LGBMClassifier(), vec2)
%time model2.fit(X_train, y_train)
model2.evaluate(X_test, y_test)

CPU times: user 3min 12s, sys: 727 ms, total: 3min 13s
Wall time: 1min 7s


/home/klui/.local/lib/python3.7/site-packages/sklearn/preprocessing/label.py:151: DeprecationWarning: The truth value of an empty array is ambiguous. Returning False, but in future this will result in an error. Use `array.size > 0` to check that an array is not empty.
  if diff:


0.9465408805031447

In [12]:
vec3 = Vectorizer(tokenizer='char', build_hash_dict=True, n_features=2**10, n_gram=(2,), hash_seed=2)
model3 = Model(lgb.LGBMClassifier(), vec3)
%time model3.fit(X_train, y_train)
model3.evaluate(X_test, y_test)

CPU times: user 3min 23s, sys: 797 ms, total: 3min 24s
Wall time: 1min 11s


/home/klui/.local/lib/python3.7/site-packages/sklearn/preprocessing/label.py:151: DeprecationWarning: The truth value of an empty array is ambiguous. Returning False, but in future this will result in an error. Use `array.size > 0` to check that an array is not empty.
  if diff:


0.9418238993710691

In [23]:
a = model1.vectorizer.hash_dict[model1.model.feature_importances_.argmax()]

In [19]:
b = model2.vectorizer.hash_dict[model2.model.feature_importances_.argmax()]

In [20]:
c = model3.vectorizer.hash_dict[model3.model.feature_importances_.argmax()]

In [21]:
a.intersection(b).intersection(c)

{';\n'}

In [34]:
from collections import defaultdict
codedict = defaultdict(list)
for m in [model1, model2, model3]:
    hd = m.vectorizer.hash_dict
    for i in range(2**10):
        for x in hd[i]:
            codedict[x].append(m.model.feature_importances_[i])

In [35]:
geo_mean = lambda x, y, z: (x*y*z)**(1/3)

In [36]:
codelist = [(x, geo_mean(*codedict[x])) for x in codedict]

In [38]:
codelist.sort(key=lambda x: x[1], reverse=True)

In [45]:
for x in [x[0] for x in codelist][:10]:
    print(repr(x))

';\n'
')\n'
');'
' ('
'in'
't '
' ='
' :'
', '
'c '


In [41]:
from pprint import pprint